# transforming mast notebooks into nbconvert templates.

this is rough, first pass of an end-to-end sketch of the MAST notebooks using dataframes and nbconvert a11y templates.

to build docs or a book from nbconvert templates we need to either shim into mkdocs
or build our own thing. sphinx can't because of how `docutils` is used.

its just easier to build our own thing. hubris, eh? lets find out.

In [1]:
    import nbconvert, nbformat, io, midgy
    from toolz.curried import *
    from nobook import *
    from IPython.display import *

In [2]:
%%
start with a string mapping the github repository to disc

    s = Series({(repo := 
https://github.com/spacetelescope/mast_notebooks
              
    ): "mast_notebooks"}).path()
    if not s.path.exists().any():
the whole repository is pretty tedious to clone. lots of pictures? do notebooks suck that bad over a long time?
better to use the `--depth` arg for control
        
        !git clone $repo --depth 1


start with a string mapping the github repository to disc

    s = Series({(repo := 
https://github.com/spacetelescope/mast_notebooks
              
    ): "mast_notebooks"}).path()
    if not s.path.exists().any():
the whole repository is pretty tedious to clone. lots of pictures? do notebooks suck that bad over a long time?
better to use the `--depth` arg for control
        
        !git clone $repo --depth 1


/home/tbone/Documents/midgy/src/midgy/_ipython.py:69: RuntimeWarning: coroutine 'InteractiveShell.run_ast_nodes' was never awaited
  get_event_loop().run_until_complete(


In [3]:
%%
this example reuses the mast notebook table of contents for jupyter book. on a mkdocs site we could do the same with mkdocs.yml.

    toc = (
        await Index(["mast_notebooks/_toc.yml"], name="path").apath().apath.load()
    ).aseries()


this example reuses the mast notebook table of contents for jupyter book. on a mkdocs site we could do the same with mkdocs.yml.

    toc = (
        await Index(["mast_notebooks/_toc.yml"], name="path").apath().apath.load()
    ).aseries()


In [4]:
    config = (
        await Index(["mast_notebooks/_config.yml"], name="path").apath().apath.load()
    ).aseries().T.iloc[:,0]

In [5]:
%%
explode the chapters, sections, files, and ultimately discover the paths of the notebooks in the documents.

    chapters = toc.parts.enumerate("chapter").aseries()
    sections = chapters.chapters.enumerate("section").aseries()   
    files = sections.sections.dropna().enumerate("section").aseries().combine_first(
        sections[["file"]].set_index(Index([0]*len(sections), name="section"), append=True)
    )
    paths = ("mast_notebooks" / files.file.apath())
    print(F"{(~paths.path().path.exists()).sum()} files missing")
    paths = paths[await paths.apath().apath.exists()].pipe(Index)
    files


explode the chapters, sections, files, and ultimately discover the paths of the notebooks in the documents.

    chapters = toc.parts.enumerate("chapter").aseries()
    sections = chapters.chapters.enumerate("section").aseries()   
    files = sections.sections.dropna().enumerate("section").aseries().combine_first(
        sections[["file"]].set_index(Index([0]*len(sections), name="section"), append=True)
    )
    paths = ("mast_notebooks" / files.file.apath())
    print(F"{(~paths.path().path.exists()).sum()} files missing")
    paths = paths[await paths.apath().apath.exists()].pipe(Index)
    files


4 files missing


file
path                    chapter section section                                                   
mast_notebooks/_toc.yml 0       0       0        notebooks/astrocut/making_tess_cubes_and_cutou...
                        1       0       0                            notebooks/astroquery/intro.md
                                1       0        notebooks/astroquery/beginner_search/beginner_...
                                        1        notebooks/astroquery/beginner_zcut/beginner_zc...
                                        2        notebooks/astroquery/large_downloads/large_dow...
...                                                                                            ...
                        10      1       2        notebooks/TESS/asteroid_rotation/asteroid_rota...
                                2       0        notebooks/TESS/interm_tesscut_dss_overlay/inte...
                                        1        notebooks/TESS/interm_tesscut_requests/interm_...
                                3       0        notebooks/TESS/interm_tess_prf_retrieve/interm...
                                        1        notebooks/TESS/removing_scattered_light_using_...

[65 rows x 1 columns]

In [6]:
    notebooks = (await paths.apath().apath.load())

filter notebooks to and currently ignore the markdown files in the mix.
the markdown files can be represents as a notebook with a markdown cell.

In [7]:
    notebooks = notebooks[notebooks.index.path.suffix.eq(".ipynb")].apply(
        nbformat.from_dict
    ).to_frame("nb")

In [8]:
%%
merge our default async jinja exporter with the legacy nbconvert importer. the async template is a massive improvement of the sync nbconvert.

    
    exporter = nbconvert.get_exporter("a11y")()
generate the default `resources` object for templating the notebook

    _, resources = exporter.from_notebook_node(nbformat.reads("""{"cells": [], "metadata": {}}""", 4))    
    ours,theirs = Series().template.environment,exporter.environment
    ours.loader = theirs.loader
    ours.filters.update({**theirs.filters, **ours.filters})
    ours.globals.update({**theirs.globals, **ours.globals})



merge our default async jinja exporter with the legacy nbconvert importer. the async template is a massive improvement of the sync nbconvert.

    
    exporter = nbconvert.get_exporter("a11y")()
generate the default `resources` object for templating the notebook

    _, resources = exporter.from_notebook_node(nbformat.reads("""{"cells": [], "metadata": {}}""", 4))    
    ours,theirs = Series().template.environment,exporter.environment
    ours.loader = theirs.loader
    ours.filters.update({**theirs.filters, **ours.filters})
    ours.globals.update({**theirs.globals, **ours.globals})


In [9]:
%% -sw
    footer = ours.filters["markdown"](F"""
By {config.author}
    
© Copyright {config.copyright}

    """)
    footer

footer = ours.filters["markdown"](F"""
By {config.author}

© Copyright {config.copyright}

""")
footer

'<p>By STScI</p>\n<p>© Copyright 2022-2024</p>\n'

In [10]:
%%
add site navigation in the post processing step
previous and next still needs to be added previous and next is relative.
we'll want the config to do this work too. it needs to be passed to the template to construct license and link information



add site navigation in the post processing step
previous and next still needs to be added previous and next is relative.
we'll want the config to do this work too. it needs to be passed to the template to construct license and link information


In [11]:
    htmls = (
        await notebooks.head().template.render_template(
            "a11y/table.html.j2", resources=resources, config=config,
            footer=footer
        )
    ).apply(exporter.post_process_html)
    htmls.to_frame().T

file,mast_notebooks/notebooks/astrocut/making_tess_cubes_and_cutouts/making_tess_cubes_and_cutouts.ipynb,mast_notebooks/notebooks/astroquery/beginner_search/beginner_search.ipynb,mast_notebooks/notebooks/astroquery/beginner_zcut/beginner_zcut.ipynb,mast_notebooks/notebooks/astroquery/large_downloads/large_downloads.ipynb,mast_notebooks/notebooks/astroquery/historic_quasar_observations/historic_quasar_observations.ipynb
0,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...","<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...","<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...","<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...","<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ..."


view the generated documentation as inline iframes

In [12]:
    iframe = """<iframe height=600 width="100%" srcdoc="{}"></iframe>"""
    iframes = htmls.apply(compose_left(__import__("html").escape, iframe.format))
    display(*iframes.apply(HTML))

/home/tbone/mambaforge/envs/p311/lib/python3.11/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## closing thoughts 

these notebooks have unexecuted code and `nbconvert-a11y` had to updated to handle that case. cf https://github.com/deathbeds/nbconvert-a11y/issues/33

currently i cannot handle site navigation~~ and licensing~~, but that is on the roadmap. (we want to think about them accessibly).
i'd like to use the jupyter toc as the toc to generate these templates.
this format for the documentation is a lot more flexible to modify than standard documentation systems.
we are dealing with our documentation as an intermediate table to offers introspection and manipulation.

search is a last thing to implement
are the colab and binder links even valid?
we'll need templates to handle myst admonitions